In [55]:
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr
from pydantic import BaseModel
from PyPDF2 import PdfReader
import json

In [56]:
load_dotenv(override=True)
openai= OpenAI()
model = "gpt-4o-mini"

In [57]:
def record_user_details(email, name="Name not provided", notes="No notes provided"):
    print(f"Recording {name} with {email} and notes: {notes}")
    return {"recorded": "ok"}

In [58]:
def record_unknown_question(question):
    print(f"Recording {question} which I could not answer.")
    return {"recorded": "ok"}

In [59]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address.",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "email address of this user"
            },
            "name": {
                "type": "string",
                "description": "name of this user if they provided it"
            },
            "notes": {
                "type": "string",
                "description": "any additional information about the conversation that is worth recording, if there is any"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [60]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Use this tool to record any question that you could not answer as you do not know the answer.",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "the question the user asked that you could not answer on your own."
                },
            },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [61]:
tools = [{"type": "function", "function": record_user_details_json},
         {"type": "function", "function": record_unknown_question_json},
         ]

In [62]:
def handle_tool_calls(tool_calls):
    results = []

    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        
        print(f"Tool called {tool_name} with arguments: {arguments}", flush=True)

        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}

        results.append({"role": "tool", "content": json.dumps(result), "tool_call_id": tool_call.id})

    return results

In [63]:
reader = PdfReader("me/about-me.pdf")
about = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        about += text
    
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Kaya ATASOY"

In [64]:

system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and Resume which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## Resume:\n{about}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [65]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    
    done = False
    while not done:
        response = openai.chat.completions.create(model=model, messages=messages, tools=tools)
        finish_reason = response.choices[0].finish_reason

        if finish_reason == "tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
        
            messages.append(message)
            messages.extend(results)
        else: done = True

    return response.choices[0].message.content
        

In [ ]:
gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


Tool called record_unknown_question with arguments: {'question': 'do you hold any patents'}
Recording do you hold any patents which I could not answer.
Tool called record_user_details with arguments: {'email': 'kaya.atsy@gmail.com'}
Recording Name not provided with kaya.atsy@gmail.com and notes: No notes provided
